In [1]:
#!/usr/bin/env python
# coding: utf-8
# In[1]:
#!/usr/bin/env python
# coding: utf-8
# get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
import glob
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold, KFold
import gc
from tqdm import tqdm_notebook, tqdm
import json
from typing import NamedTuple
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
print(torch.__version__)
# from tools import eval_summary, save_feature_importance, merge_preds
from tools2 import EarlyStopping

device = torch.device('cpu')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

/home/aiden/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


2.3.2
1.3.1
GeForce RTX 2070 SUPER


device(type='cuda', index=0)

In [2]:
# In[2]:
torch.set_num_threads(8)
torch.get_num_threads()

8

In [3]:
class SemiDataset(Dataset):
    def __init__(self, df, fea_cols, y_cols):        
        self.X = df[fea_cols].values
        self.y = df[y_cols].values
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx].astype(np.float32), self.y[idx].astype(np.float32)
    

df_train = pd.read_csv('input/train.csv', dtype=np.float32)
df_test = pd.read_csv('input/test.csv', dtype=np.float32)

print(df_train.shape, df_test.shape)

layer_cols = [c for c in df_train.columns if 'layer_' in c]
fea_cols = [c for c in df_train.columns if c not in layer_cols]

len(fea_cols), len(layer_cols)

# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(df_train[fea_cols].values)

scaler = joblib.load('scaler.bin')

df_train[fea_cols] = scaler.transform(df_train[fea_cols].values)
df_test[fea_cols] = scaler.transform(df_test[fea_cols].values)

torch.manual_seed(81511991154)
torch.initial_seed()

df_model = df_train

dataset = SemiDataset(df_model[fea_cols + layer_cols], fea_cols, layer_cols)

train_set, val_set = torch.utils.data.random_split(dataset, [700000, 110000])

print(len(train_set), len(val_set))

batch_size = 25000
num_workers = 4

train_loader = DataLoader(dataset=train_set, batch_size=batch_size, num_workers=num_workers)
val_loader = DataLoader(dataset=val_set, batch_size=22000)

print(f'batch_size {batch_size} num_workers {num_workers}')
print(f'train_loader {len(train_loader)} val_loader {len(val_loader)}')

(810000, 230) (10000, 227)
700000 110000
batch_size 25000 num_workers 4
train_loader 28 val_loader 5


#### Model

In [4]:
class Trainer(object):
    def __init__(self, model, criterion, optimizer, scheduler, device):
        self.device = device
        self.model = model#.to(self.device)
        self.criterion = criterion#.to(self.device)
        self.optimizer = optimizer
        self.scheduler = scheduler

        print(self.model.train())
        pass
    
    def set_optimizer(self, optimizer):
        self.optimizer = optimizer
        
    def set_scheduler(self, scheduler):
        self.scheduler = scheduler
        
    def train_(self, X_batch, y_batch):
        self.model.train()
        total_loss = 0
        X_batch = X_batch.to(self.device)
        y_batch = y_batch.to(self.device)

        y_pred = self.model(X_batch)
#             print(y_pred, y_batch)

        loss = self.criterion(y_pred, y_batch)
        total_loss = total_loss + loss.item()

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.scheduler.step()
        
        return total_loss / len(data_loader)
    
    def train(self, data_loader):
        self.model.train()
        total_loss = 0
        for data in data_loader:
            X_batch, y_batch = data
            X_batch = X_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            
            y_pred = self.model(X_batch)
#             print(y_pred, y_batch)
            
            loss = self.criterion(y_pred, y_batch)
            total_loss = total_loss + loss.item()

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        self.scheduler.step()
        
        return total_loss / len(data_loader)
    
    def eval(self, data_loader):
        self.model.eval()
        total_loss = 0
#         print('valid_loader', len(valid_loader))
        for data in data_loader:
            X_batch, y_batch = data
            X_batch = X_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            with torch.no_grad():
                y_pred = self.model(X_batch)
                loss = self.criterion(y_pred, y_batch)
                total_loss = total_loss + loss.item()
        return total_loss / len(data_loader)

    def save(self, model_path='checkpoint.pt'):
        joblib.dump(self.model, model_path)
        return
    
    def load(self, model_path='checkpoint.pt'):
        self.model = joblib.load(model_path)
        return

In [5]:
class DNNModel(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3):
        super(DNNModel,self).__init__()
    def forward(self, x):
        return self.model(x)
 

In [6]:
df_train = joblib.load('df_pred_train_20.pkl')
df_valid = joblib.load('df_pred_valid_20.pkl')
df_score = joblib.load('df_pred_score_20.pkl')

In [7]:
df_train

,layer_1,layer_2,layer_3,layer_4,layer_1_k0,layer_2_k0,layer_3_k0,layer_4_k0,layer_1_k1,layer_2_k1,...,layer_3_k18,layer_4_k18,layer_1_k19,layer_2_k19,layer_3_k19,layer_4_k19,layer_1_k20,layer_2_k20,layer_3_k20,layer_4_k20
id,,,,,,,,,,,,,,,,,,,,,
0,130.0,70.0,200.0,40.0,130.198746,70.655724,200.810684,38.631935,130.115753,70.740623,...,200.855026,38.728607,130.431976,70.634949,200.651657,38.688160,129.956970,70.756943,200.446228,38.579800
1,10.0,220.0,280.0,40.0,10.315645,221.055069,280.617462,40.014927,10.019632,221.140793,...,280.777069,40.120453,9.981257,221.093674,280.668060,40.074966,9.955928,221.266205,280.756287,39.994888
2,50.0,270.0,300.0,190.0,50.125122,271.359436,300.649017,189.705536,50.068932,271.536987,...,300.613892,189.493515,50.027592,271.791290,300.617065,189.941330,50.019089,271.366974,300.417023,189.476181
3,160.0,160.0,250.0,300.0,159.772964,160.222473,251.066986,299.401337,159.722519,160.169296,...,251.013794,299.296844,159.548782,160.269836,251.024155,299.587433,159.684296,160.345123,251.244400,299.396271
4,250.0,170.0,120.0,280.0,249.796127,170.280548,121.125389,279.056122,249.710541,170.399460,...,121.239616,279.121307,249.662796,169.869751,121.072014,279.133331,249.584778,170.934357,120.834770,278.934479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699995,140.0,190.0,270.0,210.0,140.023178,189.129547,269.011841,209.949219,140.080719,189.232834,...,268.900208,210.028244,139.996826,189.277451,268.986908,210.015228,139.880386,188.980667,269.155518,210.120804
699996,160.0,300.0,120.0,10.0,160.762497,298.830444,119.948357,9.569082,160.153717,298.944275,...,120.033363,9.605922,160.467361,299.095673,119.911880,9.552511,160.454407,298.916290,119.849968,9.774055
699997,270.0,300.0,250.0,150.0,269.956146,298.936646,248.333694,150.379181,270.147949,299.262848,...,248.633026,150.392899,270.107788,299.234680,248.522110,150.543884,270.156952,299.317596,248.728699,150.407516


In [11]:
class DNNModel(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3, dims=32):
        super(DNNModel,self).__init__()
        act = torch.nn.ELU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.m1 = torch.nn.Sequential(
            torch.nn.Linear(input_size, dims), torch.nn.BatchNorm1d(dims), act, dropout, 
            torch.nn.Linear(dims, dims), torch.nn.BatchNorm1d(dims), act, dropout, 
            torch.nn.Linear(dims, dims), torch.nn.BatchNorm1d(dims), act, dropout,
            torch.nn.Linear(dims, dims //2), torch.nn.BatchNorm1d(dims //2), act, dropout,
            torch.nn.Linear(dims //2, dims), torch.nn.BatchNorm1d(dims), act, dropout,
            torch.nn.Linear(dims, dims), torch.nn.BatchNorm1d(dims), act, dropout,
            torch.nn.Linear(dims, dims //2), torch.nn.BatchNorm1d(dims //2), act, dropout,
            torch.nn.Linear(dims //2, dims), torch.nn.BatchNorm1d(dims), act, dropout,
            torch.nn.Linear(dims, dims), torch.nn.BatchNorm1d(dims), act, dropout,
#             torch.nn.Linear(128, 256), torch.nn.BatchNorm1d(256), act, dropout,
#             torch.nn.Linear(256, 256), torch.nn.BatchNorm1d(256), act, dropout,
#             torch.nn.Linear(256, 256), torch.nn.BatchNorm1d(256), act, dropout,
#             torch.nn.Linear(256, 128), torch.nn.BatchNorm1d(128), act, dropout,
#             torch.nn.Linear(128, 128), torch.nn.BatchNorm1d(128), act, dropout,
#             torch.nn.Linear(128, 128), torch.nn.BatchNorm1d(128), act, dropout,
            torch.nn.Linear(dims, 1)
        )
    def forward(self, x):
        return self.m1(x)
 

In [ ]:
for dims in range(320, 513, 64):
    print('dims', dims)
    model_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
    for i in [1, 2, 3, 4]:
#     for i in [3, 4]:
        layer_cols = [f'layer_{i}']
        fea_cols = [c for c in df_train.columns if c not in layer_cols and c[6] == str(i)]
        print(fea_cols)
        print(layer_cols)

        train_set = SemiDataset(df_train, fea_cols, layer_cols)
        val_set = SemiDataset(df_valid, fea_cols, layer_cols)

        print(len(train_set), len(val_set))

#         batch_size = 700000 // 2
        batch_size = 70000
        num_workers = 8

        train_loader = DataLoader(dataset=train_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)
        val_loader = DataLoader(dataset=val_set, batch_size=110000)

        print(f'batch_size {batch_size} num_workers {num_workers}')
        print(f'train_loader {len(train_loader)} val_loader {len(val_loader)}')

        model = DNNModel(input_size=len(fea_cols), dropout_probability=0, dims=dims).to(device)

        criterion = nn.L1Loss(reduction='mean').to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=0.1)
        scheduler = StepLR(optimizer, step_size=50, gamma=1.0)
        trainer = Trainer(model, criterion, optimizer, scheduler, device)


        val_loss_min = np.Inf

        lr_list = [
            (0.01, 20),
            (0.001, 20),
            (0.0001, 20),
            (0.00003, 20),
            (0.00001, 20),
            (0.000005, 20),
        ]


        total_epoch = 10000

        for lr, patience in lr_list:
            print(lr, patience)
            if os.path.isfile('stop.flag'):
                print('stop!')
                break

            early_stopping = EarlyStopping(patience=patience, min_epoch=1, verbose=True)
            early_stopping.val_loss_min = val_loss_min
            early_stopping.best_score = None if val_loss_min==np.Inf else -val_loss_min 

        #     criterion = nn.L1Loss(reduction='mean').to(device)

            trainer.optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        #     trainer.optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
        #     trainer.optimizer = torch.optim.SGD(model.parameters(), lr=lr)

            trainer.scheduler = StepLR(trainer.optimizer, step_size=50, gamma=1.0)

            for e in tqdm_notebook(range(total_epoch), total=total_epoch, desc='Epoch'):
                if os.path.isfile('stop.flag'):
                    print(f'{e} stop!')
                    break

                train_loss = trainer.train(train_loader)
#                 train_loss = trainer.train_(X_batch[i], y_batch[i])

                if e % 1 == 0:
                    valid_loss = trainer.eval(val_loader)
            #         valid_loss = train_loss

                    ts = datetime.now().strftime('%Y%m%dT%H%M%S')
                    print(f'[{ts}] Epock {e} / {total_epoch}\t lr {trainer.scheduler.get_lr()[0]}')
                    print(f'  train_loss: {train_loss}  valid_loss: {valid_loss}')

                    early_stopping(valid_loss, model)

                    if early_stopping.early_stop:
                        print("\tEarly stopping epoch {}, valid loss {}".format(e, early_stopping.val_loss_min))
                        break

            model.load_state_dict(torch.load('model/checkpoint2.pt'))
            val_loss_min = early_stopping.val_loss_min



            print(model_ts)
            model_path = 'model/{}_e2_{}_{}_{}'.format(model_ts, dims, i, val_loss_min)
            trainer.save('{}.model'.format(model_path))
            print(model_path)

    #     break



dims 320
['layer_1_k0', 'layer_1_k1', 'layer_1_k2', 'layer_1_k3', 'layer_1_k4', 'layer_1_k5', 'layer_1_k6', 'layer_1_k7', 'layer_1_k8', 'layer_1_k9', 'layer_1_k10', 'layer_1_k11', 'layer_1_k12', 'layer_1_k13', 'layer_1_k14', 'layer_1_k15', 'layer_1_k16', 'layer_1_k17', 'layer_1_k18', 'layer_1_k19', 'layer_1_k20']
['layer_1']
700000 110000
batch_size 70000 num_workers 8
train_loader 10 val_loader 1
DNNModel(
  (m1): Sequential(
    (0): Linear(in_features=21, out_features=320, bias=True)
    (1): BatchNorm1d(320, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
    (3): Dropout(p=0, inplace=False)
    (4): Linear(in_features=320, out_features=320, bias=True)
    (5): BatchNorm1d(320, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ELU(alpha=1.0)
    (7): Dropout(p=0, inplace=False)
    (8): Linear(in_features=320, out_features=320, bias=True)
    (9): BatchNorm1d(320, eps=1e-05, momentum=0.1, affine=True, track_running_stat

[20200202T201933] Epock 0 / 10000	 lr 0.01
  train_loss: 151.38211517333986  valid_loss: 79.62552642822266
Validation loss decreased (inf --> 79.62552643).  Saving model ...
[20200202T201938] Epock 1 / 10000	 lr 0.01
  train_loss: 138.93831787109374  valid_loss: 160.4217529296875
EarlyStopping 2 / 1 counter: 1 out of 20
[20200202T201942] Epock 2 / 10000	 lr 0.01
  train_loss: 117.23819427490234  valid_loss: 105.99746704101562
EarlyStopping 3 / 1 counter: 2 out of 20
[20200202T201947] Epock 3 / 10000	 lr 0.01
  train_loss: 85.35431213378907  valid_loss: 75.789306640625
Validation loss decreased (79.62552643 --> 75.78930664).  Saving model ...
[20200202T201951] Epock 4 / 10000	 lr 0.01
  train_loss: 38.74831485748291  valid_loss: 92.26327514648438
EarlyStopping 5 / 1 counter: 1 out of 20
[20200202T201956] Epock 5 / 10000	 lr 0.01
  train_loss: 14.027966976165771  valid_loss: 150.5469512939453
EarlyStopping 6 / 1 counter: 2 out of 20
[20200202T202001] Epock 6 / 10000	 lr 0.01
  train_loss

[20200202T202331] Epock 52 / 10000	 lr 0.01
  train_loss: 1.1778605341911317  valid_loss: 6.6289873123168945
EarlyStopping 53 / 1 counter: 8 out of 20
[20200202T202336] Epock 53 / 10000	 lr 0.01
  train_loss: 1.0610235929489136  valid_loss: 5.954131603240967
EarlyStopping 54 / 1 counter: 9 out of 20
[20200202T202341] Epock 54 / 10000	 lr 0.01
  train_loss: 1.0722674548625946  valid_loss: 7.095727443695068
EarlyStopping 55 / 1 counter: 10 out of 20
[20200202T202345] Epock 55 / 10000	 lr 0.01
  train_loss: 0.9347982048988343  valid_loss: 4.49274206161499
EarlyStopping 56 / 1 counter: 11 out of 20
[20200202T202350] Epock 56 / 10000	 lr 0.01
  train_loss: 1.1579614579677582  valid_loss: 4.981287002563477
EarlyStopping 57 / 1 counter: 12 out of 20
[20200202T202354] Epock 57 / 10000	 lr 0.01
  train_loss: 1.476398777961731  valid_loss: 6.16628360748291
EarlyStopping 58 / 1 counter: 13 out of 20
[20200202T202359] Epock 58 / 10000	 lr 0.01
  train_loss: 1.2727836310863494  valid_loss: 4.730076

[20200202T202739] Epock 106 / 10000	 lr 0.01
  train_loss: 1.2849928140640259  valid_loss: 3.8652846813201904
EarlyStopping 107 / 1 counter: 11 out of 20
[20200202T202743] Epock 107 / 10000	 lr 0.01
  train_loss: 1.2043302178382873  valid_loss: 4.910299301147461
EarlyStopping 108 / 1 counter: 12 out of 20
[20200202T202748] Epock 108 / 10000	 lr 0.01
  train_loss: 1.0315131366252899  valid_loss: 4.853309154510498
EarlyStopping 109 / 1 counter: 13 out of 20
[20200202T202753] Epock 109 / 10000	 lr 0.01
  train_loss: 0.8649782061576843  valid_loss: 6.647603511810303
EarlyStopping 110 / 1 counter: 14 out of 20
[20200202T202757] Epock 110 / 10000	 lr 0.01
  train_loss: 0.8958196759223938  valid_loss: 8.658158302307129
EarlyStopping 111 / 1 counter: 15 out of 20
[20200202T202802] Epock 111 / 10000	 lr 0.01
  train_loss: 1.1027437686920165  valid_loss: 7.1371307373046875
EarlyStopping 112 / 1 counter: 16 out of 20
[20200202T202806] Epock 112 / 10000	 lr 0.01
  train_loss: 0.941568261384964  va

[20200202T202825] Epock 0 / 10000	 lr 0.001
  train_loss: 0.9901831090450287  valid_loss: 3.9694113731384277
EarlyStopping 1 / 1 counter: 1 out of 20
[20200202T202830] Epock 1 / 10000	 lr 0.001
  train_loss: 0.8101999759674072  valid_loss: 4.634033679962158
EarlyStopping 2 / 1 counter: 2 out of 20
[20200202T202834] Epock 2 / 10000	 lr 0.001
  train_loss: 0.7045645117759705  valid_loss: 7.8411102294921875
EarlyStopping 3 / 1 counter: 3 out of 20
[20200202T202839] Epock 3 / 10000	 lr 0.001
  train_loss: 0.6473161697387695  valid_loss: 8.07517147064209
EarlyStopping 4 / 1 counter: 4 out of 20
[20200202T202843] Epock 4 / 10000	 lr 0.001
  train_loss: 0.6050409018993378  valid_loss: 9.085381507873535
EarlyStopping 5 / 1 counter: 5 out of 20
[20200202T202848] Epock 5 / 10000	 lr 0.001
  train_loss: 0.6586878955364227  valid_loss: 9.18789005279541
EarlyStopping 6 / 1 counter: 6 out of 20
[20200202T202852] Epock 6 / 10000	 lr 0.001
  train_loss: 0.6997353613376618  valid_loss: 5.73135614395141

...

[20200202T204454] Epock 52 / 10000	 lr 0.01
  train_loss: 1.401772689819336  valid_loss: 3.1641688346862793
EarlyStopping 53 / 1 counter: 1 out of 20
[20200202T204459] Epock 53 / 10000	 lr 0.01
  train_loss: 1.020945143699646  valid_loss: 2.5048818588256836
EarlyStopping 54 / 1 counter: 2 out of 20
[20200202T204503] Epock 54 / 10000	 lr 0.01
  train_loss: 1.4564532697200776  valid_loss: 7.074245929718018
EarlyStopping 55 / 1 counter: 3 out of 20
[20200202T204508] Epock 55 / 10000	 lr 0.01
  train_loss: 1.5052939534187317  valid_loss: 5.008861541748047
EarlyStopping 56 / 1 counter: 4 out of 20
[20200202T204513] Epock 56 / 10000	 lr 0.01
  train_loss: 1.3484441459178924  valid_loss: 5.42759370803833
EarlyStopping 57 / 1 counter: 5 out of 20
[20200202T204517] Epock 57 / 10000	 lr 0.01
  train_loss: 1.019835740327835  valid_loss: 6.771763801574707
EarlyStopping 58 / 1 counter: 6 out of 20
[20200202T204522] Epock 58 / 10000	 lr 0.01
  train_loss: 1.3821391761302948  valid_loss: 2.8596770763

[20200202T204716] Epock 0 / 10000	 lr 0.001
  train_loss: 0.9721040844917297  valid_loss: 2.9538040161132812
EarlyStopping 1 / 1 counter: 1 out of 20
[20200202T204721] Epock 1 / 10000	 lr 0.001
  train_loss: 0.80233433842659  valid_loss: 5.000492095947266
EarlyStopping 2 / 1 counter: 2 out of 20
[20200202T204726] Epock 2 / 10000	 lr 0.001
  train_loss: 0.8148698747158051  valid_loss: 5.674340724945068
EarlyStopping 3 / 1 counter: 3 out of 20
[20200202T204730] Epock 3 / 10000	 lr 0.001
  train_loss: 0.8419364273548127  valid_loss: 6.789543151855469
EarlyStopping 4 / 1 counter: 4 out of 20
[20200202T204735] Epock 4 / 10000	 lr 0.001
  train_loss: 0.7653651893138885  valid_loss: 5.992943286895752
EarlyStopping 5 / 1 counter: 5 out of 20
[20200202T204739] Epock 5 / 10000	 lr 0.001
  train_loss: 0.7157977044582366  valid_loss: 4.623096942901611
EarlyStopping 6 / 1 counter: 6 out of 20
[20200202T204744] Epock 6 / 10000	 lr 0.001
  train_loss: 0.7428443729877472  valid_loss: 5.32178258895874


[20200202T204848] Epock 0 / 10000	 lr 0.0001
  train_loss: 0.7822809398174286  valid_loss: 1.0791817903518677
Validation loss decreased (1.15208948 --> 1.07918179).  Saving model ...
[20200202T204852] Epock 1 / 10000	 lr 0.0001
  train_loss: 0.6853814840316772  valid_loss: 1.237899899482727
EarlyStopping 2 / 1 counter: 1 out of 20
[20200202T204857] Epock 2 / 10000	 lr 0.0001
  train_loss: 0.6806141436100006  valid_loss: 0.8812962770462036
Validation loss decreased (1.07918179 --> 0.88129628).  Saving model ...
[20200202T204901] Epock 3 / 10000	 lr 0.0001
  train_loss: 0.6652186512947083  valid_loss: 0.9286168217658997
EarlyStopping 4 / 1 counter: 1 out of 20
[20200202T204906] Epock 4 / 10000	 lr 0.0001
  train_loss: 0.7180307269096374  valid_loss: 0.9329356551170349
EarlyStopping 5 / 1 counter: 2 out of 20
[20200202T204910] Epock 5 / 10000	 lr 0.0001
  train_loss: 0.6973221600055695  valid_loss: 0.7533663511276245
Validation loss decreased (0.88129628 --> 0.75336635).  Saving model ...

[20200202T205046] Epock 0 / 10000	 lr 3e-05
  train_loss: 0.6781978487968445  valid_loss: 0.8464998602867126
EarlyStopping 1 / 1 counter: 1 out of 20
[20200202T205051] Epock 1 / 10000	 lr 3e-05
  train_loss: 0.6591175317764282  valid_loss: 0.8239643573760986
EarlyStopping 2 / 1 counter: 2 out of 20
[20200202T205055] Epock 2 / 10000	 lr 3e-05
  train_loss: 0.6899183392524719  valid_loss: 0.7120437622070312
Validation loss decreased (0.75336635 --> 0.71204376).  Saving model ...
[20200202T205100] Epock 3 / 10000	 lr 3e-05
  train_loss: 0.6785852670669555  valid_loss: 0.6459470987319946
Validation loss decreased (0.71204376 --> 0.64594710).  Saving model ...
[20200202T205104] Epock 4 / 10000	 lr 3e-05
  train_loss: 0.6490680634975433  valid_loss: 0.6816381216049194
EarlyStopping 5 / 1 counter: 1 out of 20
[20200202T205109] Epock 5 / 10000	 lr 3e-05
  train_loss: 0.6636123061180115  valid_loss: 0.7138848304748535
EarlyStopping 6 / 1 counter: 2 out of 20
[20200202T205113] Epock 6 / 10000	 l

[20200202T205317] Epock 0 / 10000	 lr 1e-05
  train_loss: 0.6893961369991303  valid_loss: 0.5979162454605103
Validation loss decreased (0.62169015 --> 0.59791625).  Saving model ...
[20200202T205321] Epock 1 / 10000	 lr 1e-05
  train_loss: 0.6567171096801758  valid_loss: 0.6217405796051025
EarlyStopping 2 / 1 counter: 1 out of 20
[20200202T205326] Epock 2 / 10000	 lr 1e-05
  train_loss: 0.6668659329414368  valid_loss: 0.6063059568405151
EarlyStopping 3 / 1 counter: 2 out of 20
[20200202T205331] Epock 3 / 10000	 lr 1e-05
  train_loss: 0.6887971639633179  valid_loss: 0.6202292442321777
EarlyStopping 4 / 1 counter: 3 out of 20
[20200202T205335] Epock 4 / 10000	 lr 1e-05
  train_loss: 0.6885318756103516  valid_loss: 0.5904062390327454
Validation loss decreased (0.59791625 --> 0.59040624).  Saving model ...
[20200202T205340] Epock 5 / 10000	 lr 1e-05
  train_loss: 0.6498657822608948  valid_loss: 0.5956825017929077
EarlyStopping 6 / 1 counter: 1 out of 20
[20200202T205344] Epock 6 / 10000	 l

In [ ]:
# e_models =[
#     'model/20200131T225354_e1_0.32568883895874023.model',
#     'model/20200131T225354_e2_0.5253744721412659.model',
#     'model/20200131T225354_e3_0.3913254737854004.model',
#     'model/20200131T225354_e4_0.3752939999103546.model',
# ]

# for i in range(1, 5):
#     layer_cols = [f'layer_{i}']
#     fea_cols = [c for c in df_score.columns if c not in layer_cols and c[-1] == str(i)]
#     print(fea_cols)
#     print(layer_cols)
#     y_pred = joblib.load(e_models[i-1])(torch.Tensor(df_score[fea_cols].values).to(device))    
#     df_score[layer_cols[0]] = y_pred.cpu().detach().numpy()
#     print(df_score[layer_cols + fea_cols])



In [ ]:
# e_models = [
#     'model/e_1_20200131T202847_0.322307288646698.model',
# #     'model/e_1_20200131T221837_0.33561456203460693.model',
#     'model/e_2_20200131T213251_0.5193350911140442.model',
#     'model/e_3_20200131T214620_0.38977283239364624.model',
#     'model/e_4_20200131T211402_0.37712812423706055.model',
# #     'model/e_4_20200131T220102_0.3780273497104645.model',
# #     'model/e_3_20200131T214620_0.38977283239364624.model',
# #     'model/e_4_20200131T211402_0.37712812423706055.model',
# #     'model/e_3_20200131T205640_0.3976164758205414.model',
# # #     'model/e_2_20200131T204154_0.5304651260375977.model',
# #     'model/e_1_20200131T202847_0.322307288646698.model',
# ]
# for i in range(1, 5):
#     layer_cols = [f'layer_{i}']
#     fea_cols = [c for c in df_score.columns if c not in layer_cols and c[-1] == str(i)]
#     print(fea_cols)
#     print(layer_cols)
#     y_pred = joblib.load(e_models[i-1])(torch.Tensor(df_score[fea_cols].values).to(device))    
#     df_score[layer_cols[0]] = y_pred.cpu().detach().numpy()
#     print(df_score[layer_cols + fea_cols])

In [ ]:
# df_score.index.name = 'id'
# layer_cols = [c for c in df_score.columns if 'la' in c]
# ts = datetime.now().strftime('%Y%m%dT%H%M%S')
# df_score[layer_cols].to_csv(f'submit/{ts}_e.csv')

In [ ]:


# #### Pred Train

# criterion = nn.L1Loss(reduction='mean').to(device)
# model.train()
# total_loss = 0
# preds = []
# y_true = []
# for data in train_loader:
#     X_batch, y_batch = data
#     X_batch = X_batch.to(device)
#     y_batch = y_batch.to(device)
#     y_true.extend(y_batch)
    
#     with torch.no_grad():
#         y_pred = model(X_batch)
#         preds.extend(y_pred)
#         loss = criterion(y_pred, y_batch)
#         total_loss = total_loss + loss.item()
        
# train_loss_min = total_loss / len(train_loader)
# train_loss_min

# y_true

# preds

# #### Pred Valid

# criterion = nn.L1Loss(reduction='mean').to(device)
# model.eval()
# total_loss = 0
# for data in val_loader:
#     X_batch, y_batch = data
#     X_batch = X_batch.to(device)
#     y_batch = y_batch.to(device)
    
#     with torch.no_grad():
#         y_pred = model(X_batch)
#         loss = criterion(y_pred, y_batch)
#         total_loss = total_loss + loss.item()
        
# val_loss_min = total_loss / len(val_loader)
# val_loss_min

# #### Pred Test 

# model.eval()
# y_pred = model(torch.Tensor(df_test[fea_cols].values).to(device))    
# print(y_pred)

# ts = datetime.now().strftime('%Y%m%dT%H%M%S')

# df_submit = pd.read_csv('input/sample_submission.csv', index_col=0)

# df_submit[layer_cols] = y_pred.cpu().detach().numpy()
# df_submit.to_csv(f'submit/{ts}_{val_loss_min}.csv')

# print(ts, val_loss_min)

# # model.load_state_dict(torch.load('model/20200123T215856_0.6811393996079763.model'))

